In [174]:
!pip install anomalies

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [175]:
import numpy as np
from anomalies import anomaly
import itertools
import random
from itertools import permutations
import sys
import time
import warnings # ignora los warnings durante compilación
warnings.filterwarnings("ignore")# ignora los warnings durante compilación
from itertools import permutations
import dask.array as da
import pandas as pd
##########################################

z=anomaly.free

def valor_m(n):
    """
    valor_m muestra el la cantidad de L y K que se produciran 
    """
    if n%2==0:
        m = n/2 -1
    else:
        m = (n-3)/2
    return m

In [176]:
def vector_like(n):
    """
    Esta funcion devuelve los vetoreslikes v_mas y v_menos o u_mas y u_menos
    a partir de los arreglos L y K tomados de la lista aleatoria con valores
    entre -15 y 15
    """
    m = int(valor_m(n))
   
#    print(lista)
     
    if n%2 == 0:
        
        lista = da.random.randint(-15, 15, (4000000, 2*m)) 
        #lista = np.unique(lista, axis=0)
        lista = lista.to_dask_dataframe().drop_duplicates().to_dask_array()
        #L = lista[:m]
        #K = lista[m:(2*m)]

        Z_sol=lista
                
    else:
        lista = da.random.randint(-15, 15, (4000000, 2*m+1)) 
        #lista = np.unique(lista, axis=0)
        lista = lista.to_dask_dataframe().drop_duplicates().to_dask_array()
        #L = lista[:m]
        #K = lista[m:(2*m)+1]

        Z_sol=lista              
    return Z_sol

In [177]:
def _get_chiral(q,q_max=np.inf):
    #Normalize to positive minimum
    if 0 in q:
        return None,None  #Quita los ceros
    if q[0]<0:
        q=-q
    #Divide by GCD
    GCD=np.gcd.reduce(q)
    q=(q/GCD).astype(int)
    if ( 0 not in [ sum(p) for p in itertools.permutations(q, 2) ] and np.abs(q).max()<=q_max):
        return q,GCD #not 0 in q, #avoid vector-like and multiple 0's ##De todos los elementos 
                      #de q, tome el más grande y positivo y menor que infinito
    else:
        return None,None

In [178]:
def get_solution(l,k,zmax=32): #aquí me devuelve q como un diccionario
    q,gcd=_get_chiral( z(l,k) )
    #if q is not None and np.abs(q).max()<=zmax:#
    if q is not None and np.abs(q).max()<=zmax:
        return {'l':l,'k':k,'z':list(q),'gcd':gcd}
    else:
        return {}    

In [179]:
def get_solution_from_list(lk,zmax=32):
    n=len(lk)
    l=lk[:n//2]
    k=lk[n//2:]
    return get_solution(l,k,zmax)


assert get_solution_from_list([1,2,1,-2])['z']==[1, 1, 1, -4, -4, 5]

In [180]:
from multiprocessing import Pool
from multiprocessing import cpu_count

In [181]:
n = int(input('inserte un valor para n:'))
m = int(valor_m(n))
print(f'el valor para m es:{m}')
Z_sol = (vector_like(n))
print(f'El arreglo vectorlike es:{Z_sol}')

df=pd.DataFrame()
s=time.time()
Z_sol=Z_sol.compute()
print('grid → ',time.time()-s,Z_sol.shape)

s=time.time()
pool = Pool(cpu_count())
proceso = pool.map(get_solution_from_list, Z_sol)
pool.close()
del Z_sol
    
proceso=[d for d in proceso if d]
print('sols → ',time.time()-s,len(proceso))

inserte un valor para n:5
el valor para m es:1
El arreglo vectorlike es:dask.array<values, shape=(nan, 3), dtype=int64, chunksize=(nan, 3), chunktype=numpy.ndarray>
grid →  0.7384274005889893 (27000, 3)
sols →  4.60492467880249 2054


In [182]:
#Unique solutions
df=df.append( proceso )#, ignore_index=True    )  
df.sort_values('gcd')
df['zs']=df['z'].astype(str)
df=df.drop_duplicates('zs').drop('zs',axis='columns').reset_index(drop=True)
print('unique solutions → ',df.shape)


unique solutions →  (12, 4)
